In [1]:

import torch
from yolov5.detect import run as yolov5_run
from utils_main.utils import ResizeImages
from siamese_triplet.datasets import LeopardDataset
from siamese_triplet.networks import EmbeddingNet
from siamese_triplet.networks import EmbeddingWithSoftmaxNet
from siamese_triplet.networks import MultiPartEmbeddingNet
from siamese_triplet.networks import MultiPartEmbeddingWithSoftmaxNet
from siamese_triplet.networks import extract_embeddings
from siamese_triplet.networks import InferenceNetwork

In [2]:
#Yolov5 folders
CROP_FOLDER = '../dataset/'
YOLOV5_FOLDER = './yolov5/'
YOLOV5_WEIGHTS_FOLDER = YOLOV5_FOLDER + 'yolov5x_weights/best.pt'
INPUT_IMAGE_FOLDER = '/home/user1/work/W210/datasets/shared/images/'

# Siamese network files/folder
model_file_name = 'leop_id_model_july_28.pt'
centroid_file_name = 'centroid_july_28.pt'
dict_file_name = 'leopard_label_dict.pt'
model_path = f"./siamese_triplet/weights/{model_file_name}"    
centroid_path = f"./siamese_triplet/weights/{centroid_file_name}"   
labels_dict_path = f"./siamese_triplet/weights/{dict_file_name}" 

# input path to identification
RESIZE_IMAGE_PATH = CROP_FOLDER+'resize/'
   

In [3]:
# Run Yolov5 to generate cropped images
yolov5_run(weights=YOLOV5_WEIGHTS_FOLDER,
    conf_thres= 0.75,
    nosave=True,
    data = YOLOV5_FOLDER+'/data/custom.yaml',
    project = CROP_FOLDER,
    name = '',
    source = INPUT_IMAGE_FOLDER,
    save_crop=True)

YOLOv5 🚀 2022-7-27 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA TITAN RTX, 24212MiB)

Fusing layers... 
Model summary: 444 layers, 86193601 parameters, 0 gradients
image 1/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_121_002352.jpg: 352x640 1 flank, 1 head, 1 leopard, Done. (0.022s)
image 2/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_12_000315.jpg: 480x640 1 head, Done. (0.025s)
image 3/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_12_000409.jpg: 480x640 1 head, Done. (0.031s)
image 4/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_12_000536.jpg: 480x640 1 head, 1 leopard, Done. (0.025s)
image 5/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_12_000552.jpg: 480x640 1 flank, 1 head, 1 leopard, Done. (0.026s)
image 6/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_12_000555.jpg: 448x640 Done. (0.024s)
image 7/19 /media/user1/My4TBHD1/W210/datasets/temp/leop_153_002482.jpg: 384x640 1 flank, 1 head, 1 leopard, Done. (0.028s)
image 8/19 /media/user1/My4TBHD1/W210/datasets/temp/l

In [ ]:
# Generate resize images from cropped images
resize_images = ResizeImages(in_path=CROP_FOLDER+'crops/',out_path=RESIZE_IMAGE_PATH+'leop_xx')
resize_images.run()

In [ ]:
# Initialize Siamese network model and load weights
MULTI_EMBEDDING = True
softmax = True
cuda = torch.cuda.is_available()

model = EmbeddingWithSoftmaxNet(num_classes=64)
if cuda:
    model.cuda()

model.load_state_dict(torch.load(model_path))
model.centroids = torch.load(centroid_path)
labels_dict = torch.load(labels_dict_path)


In [ ]:
#Run identification Network

infer_network = InferenceNetwork(infer_path=RESIZE_IMAGE_PATH,
                                 model = model,
                                 labels_dict = labels_dict,
                                 infer_labels = True,
                                 ood_reject_thresh = 0.675,
                                 cuda = cuda
                                 )
ref_label, pred_label = infer_network.run()


In [ ]:
for i in range(len(pred_label)):
   print(ref_label[i], pred_label[i])